In [2]:
# Add relative path to ExoJulia
push!(LOAD_PATH, "../../../ExoJulia/")

# import
using ExoJulia

In [3]:
using LsqFit
using PyPlot

In [4]:
# From time to M
M(t::Array,t_p::Float64,P::Float64) = (2*pi/P).*(t.-t_p)
M(t::Float64,t_p::Float64,P::Float64) = (2*pi/P)*(t-t_p)

M (generic function with 2 methods)

In [5]:
#How to convert from E to f
EtoF(E::Float64,ecc::Float64) = 2.*atan((((1.+ecc)/(1.-ecc))^(1./2.))*tan(E/2.))
EtoF(E::Array,ecc::Float64) = 2.*atan((((1.+ecc)/(1.-ecc))^(1./2.)).*tan(E./2.))

EtoF (generic function with 2 methods)

In [6]:
K(h::Float64,c::Float64) = sqrt(h*h+c*c)

K (generic function with 1 method)

In [7]:
pomega(h::Float64,c::Float64) = atand(-c/h)

pomega (generic function with 1 method)

In [8]:
gamma(v0::Float64,K::Float64,ecc::Float64,pomega::Float64) = v0-K*ecc*cosd(pomega)

gamma (generic function with 1 method)

In [9]:
function P_guesser(x_data::Array,y_data::Array)
    #minimum period to look for is distance between data points. Max is size of data. Step is 0.5
    P_array::Array{Float64} = collect(mean(diff(x_data)):0.5:maximum(x_data)-minimum(y_data))
    sq_array::Array{Float64} = [sum(diff(y_data[sortperm(mod(x_data,P))]).^2.0) for P=P_array]
    return P_array[findmin(sq_array)[2]]
end

P_guesser (generic function with 1 method)

In [10]:
data = readdlm("mystery_planet.txt",Float64) ;

In [13]:
times = data[:,1]
RVs = data[:,2]
RV_errs = data[:,3] ;

In [14]:
function kepler_solve!(Ms::Array{Float64},ecc::Float64)
    results = Float64[]
    for em in Ms
        push!(results,ExoJulia.Orbit.kepler_solve(em, ecc))
    end
    return results
end

kepler_solve! (generic function with 1 method)

In [15]:
function hcv0(f::Array{Float64},y_data::Array{Float64},y_error::Array{Float64})
    W = diagm(1.0./(y_error.^2.0))
    F = hcat(cos(f),sin(f),[1.0 for x in f])'
    epsilon = inv(F*W*F')
    return y_data'*W*F'*epsilon
end

hcv0 (generic function with 1 method)

In [16]:
function fit_RV(x_data::Array{Float64},y_data::Array{Float64},y_err::Array{Float64})
    
    function v_rad(t::Array{Float64},params::Array{Float64}) #params are e,t_p,P
        if 0 <= params[1] < 1
            Es::Array{Float64} = kepler_solve!(M(t,params[2],params[3]),params[1])
            f::Array{Float64} = EtoF(Es,params[1])
            h,c,v0 = hcv0(f,y_data,y_err)
            return h.*cos(f)+c*sin(f).+v0
        else
            return Inf
        end
    end
    
    function get_params(fit_obj::LsqFit.LsqFitResult{Float64}) #returns e,t_p,P,K,pomega,gamma
        params::Array = fit_obj.param
        Es::Array = kepler_solve!(M(x_data,params[2],params[3]), params[1])
        f::Array = EtoF(Es,params[1])
        h,c,v0 = hcv0(f,y_data,y_err)
        return params[1],params[2],params[3],K(h,c),pomega(h,c),gamma(v0,K(h,c),params[1], pomega(h,c))
    end
        
    return get_params(curve_fit(v_rad,x_data,y_data,(1.0./y_err.^2.0),[0.1,x_data[1],P_guesser(x_data,y_data)]))
    
end

fit_RV (generic function with 1 method)

In [18]:
@time result = fit_RV(times,RVs,RV_errs) ;

  0.191881 seconds (346.02 k allocations: 96.684 MB, 4.20% gc time)


In [19]:
param_names = ["e","t_p","P","K","pomega","gamma"]
for (param,res) in zip(param_names,result)
    println("$param = $res")
end

e = 0.40016612320726774
t_p = 13891.356528538307
P = 116.68569506349166
K = 316.1090092004665
pomega = -0.7766860911621005
gamma = -51.916669565780765


In [20]:
function eff_func(t,t0::Float64,r0::Float64,sma::Float64,ecc::Float64,t_p::Float64,P::Float64)
    Ms = M(t,t_p,P)
    M0 = M(t0,t_p,P)
    E0 = kepler_solve!(M0,ecc)
    E = kepler_solve!(Ms,ecc)
    return (sma/r0).*(cos(E.-E0).+1)
end

eff_func (generic function with 1 method)

In [21]:
function gee_func(t,t0::Float64,r0::Float64,sma::Float64,ecc::Float64,t_p::Float64,P::Float64)
    Ms = M(t,t_p,P)
    M0 = M(t0,t_p,P)
    E0 = kepler_solve!(M0,ecc)
    E = kepler_solve!(Ms,ecc)
    return (t.-t0).+(P/(2.0*pi)).*(sin(E.-E0)-(E.-E0))
end

gee_func (generic function with 1 method)